# Traffic assignment logit SUE (min)

$$
P_k =
\frac{\exp(-\mu \beta_{\text{time}} C_k)}
{\sum_{j \in \mathcal{K}_{od}} \exp(-\mu \beta_{\text{time}} C_j)}
$$


k_paths=5  
mu=100.0  
beta_time=1.0  
max_iter=500  
tol=1e-6  

In [3]:
import math
import numpy as np
import pandas as pd
import networkx as nx

# ----------------------------
# 1) Read & normalize inputs
# ----------------------------
def read_net_csv(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()


    if ";" in df.columns:
        df = df.drop(columns=[";"])


    required = ["A", "B", "capacity", "t0_min", "bpr_B", "bpr_power"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"net.csv missing columns: {missing}. got: {df.columns.tolist()}")

    out = pd.DataFrame()
    out["u"] = df["A"].astype(int)
    out["v"] = df["B"].astype(int)
    out["cap"] = df["capacity"].astype(float)
    out["t0"] = df["t0_min"].astype(float)
    out["alpha"] = df["bpr_B"].astype(float)       # BPR alpha, usually 0.15
    out["beta"] = df["bpr_power"].astype(float)    # BPR beta, usually 4

    return out


def read_od_csv(path: str) -> pd.DataFrame:
    od = pd.read_csv(path)
    od.columns = od.columns.str.strip()


    col_map = {}
    for c in od.columns:
        lc = c.lower()
        if lc in ["o", "origin"]:
            col_map[c] = "origin"
        elif lc in ["d", "dest", "destination"]:
            col_map[c] = "destination"
        elif lc in ["ton", "demand", "trips", "q"]:
            col_map[c] = "demand"
    od = od.rename(columns=col_map)

    required = ["origin", "destination", "demand"]
    missing = [c for c in required if c not in od.columns]
    if missing:
        raise ValueError(f"od.csv missing columns: {missing}. got: {od.columns.tolist()}")

    od = od[required].copy()
    od["origin"] = od["origin"].astype(int)
    od["destination"] = od["destination"].astype(int)
    od["demand"] = od["demand"].astype(float)

    # 去掉无效行
    od = od[(od["demand"] > 0) & (od["origin"] != od["destination"])].reset_index(drop=True)
    return od

# ----------------------------
# 2) BPR & Logit
# ----------------------------
def bpr_time(t0, x, cap, alpha=0.15, beta=4.0):
    if cap <= 0:
        return float("inf")
    return t0 * (1.0 + alpha * (x / cap) ** beta)

def logit_probs(utilities, mu=1.0):
    umax = max(utilities)
    exps = [math.exp(mu * (u - umax)) for u in utilities]
    s = sum(exps)
    return [e / s for e in exps]

# ----------------------------
# 3) K shortest paths
# ----------------------------
def k_shortest_edge_paths(G, origin, dest, k=10, weight="weight"):
    try:
        gen = nx.shortest_simple_paths(G, origin, dest, weight=weight)
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        return []

    paths = []
    for _ in range(k):
        try:
            node_path = next(gen)
        except StopIteration:
            break
        edge_path = list(zip(node_path[:-1], node_path[1:]))
        paths.append(edge_path)
    return paths

# ----------------------------
# 4) Logit-SUE via MSA
# ----------------------------
def run_logit_sue(net: pd.DataFrame,
                  od: pd.DataFrame,
                  k_paths=10,
                  mu=1.0,
                  beta_time=1.0,
                  max_iter=60,
                  tol=1e-5):

    # Edge list and indexing
    edges = list(zip(net["u"].tolist(), net["v"].tolist()))
    edge_idx = {(u, v): i for i, (u, v) in enumerate(edges)}

    # flows
    x = {e: 0.0 for e in edges}

    # graph
    G = nx.DiGraph()
    for (u, v), t0 in zip(edges, net["t0"].tolist()):
        G.add_edge(u, v, weight=float(t0))

    def compute_edge_costs(x_dict):
        c = {}
        for i, (u, v) in enumerate(edges):
            row = net.iloc[i]
            c[(u, v)] = bpr_time(row["t0"], x_dict[(u, v)], row["cap"], row["alpha"], row["beta"])
        return c

    def rel_change(x_old, x_new):
        num = sum(abs(x_new[e] - x_old[e]) for e in edges)
        den = max(1.0, sum(abs(x_old[e]) for e in edges))
        return num / den

    for it in range(1, max_iter + 1):
        # 1) update costs
        edge_costs = compute_edge_costs(x)
        for (u, v), cost in edge_costs.items():
            G[u][v]["weight"] = float(cost)

        # 2) auxiliary flow via logit loading
        x_hat = {e: 0.0 for e in edges}

        for _, r in od.iterrows():
            o = int(r["origin"])
            d = int(r["destination"])
            q = float(r["demand"])
            if q <= 0:
                continue

            cand_paths = k_shortest_edge_paths(G, o, d, k=k_paths, weight="weight")
            if not cand_paths:
                continue

            # utility: U = - beta_time * sum(cost)
            utils = []
            for p in cand_paths:
                cost_p = sum(edge_costs[e] for e in p)
                utils.append(-beta_time * cost_p)

            probs = logit_probs(utils, mu=mu)

            for p, pk in zip(cand_paths, probs):
                fp = q * pk
                for e in p:
                    x_hat[e] += fp

        # 3) MSA update
        step = 1.0 / it
        x_new = {e: x[e] + step * (x_hat[e] - x[e]) for e in edges}

        gap = rel_change(x, x_new)
        x = x_new
        print(f"Iter {it:02d} | step={step:.4f} | rel_change={gap:.6e}")

        if gap < tol:
            break

    # output
    out = net.copy()
    out["flow"] = [x[(u, v)] for (u, v) in edges]
    out["time"] = [
        bpr_time(out.loc[i, "t0"], out.loc[i, "flow"], out.loc[i, "cap"], out.loc[i, "alpha"], out.loc[i, "beta"])
        for i in range(len(out))
    ]
    return out

# ----------------------------
# 5) main
# ----------------------------
if __name__ == "__main__":
    net = read_net_csv("/Users/keito-shiraki/traffic_assignment/data/csv/SiouxFalls_net_Vff60_with_minutes.csv")
    od = read_od_csv("/Users/keito-shiraki/traffic_assignment/data/csv/SiouxFalls_od.csv")

    result = run_logit_sue(
        net=net,
        od=od,
        k_paths=5,
        mu=100.0,
        beta_time=1.0,
        max_iter=500,
        tol=1e-6
    )

    result.to_csv("assignment_logit_sue_min_result2.csv", index=False)
    print("Saved: assignment_logit_sue_min_result2.csv")

Iter 01 | step=1.0000 | rel_change=8.881000e+05
Iter 02 | step=0.5000 | rel_change=6.459858e-01
Iter 03 | step=0.3333 | rel_change=2.993324e-01
Iter 04 | step=0.2500 | rel_change=1.543137e-01
Iter 05 | step=0.2000 | rel_change=8.183730e-02
Iter 06 | step=0.1667 | rel_change=5.534300e-02
Iter 07 | step=0.1429 | rel_change=3.271276e-02
Iter 08 | step=0.1250 | rel_change=2.697268e-02
Iter 09 | step=0.1111 | rel_change=2.783637e-02
Iter 10 | step=0.1000 | rel_change=2.472281e-02
Iter 11 | step=0.0909 | rel_change=1.868793e-02
Iter 12 | step=0.0833 | rel_change=1.361003e-02
Iter 13 | step=0.0769 | rel_change=1.604433e-02
Iter 14 | step=0.0714 | rel_change=1.551080e-02
Iter 15 | step=0.0667 | rel_change=1.177913e-02
Iter 16 | step=0.0625 | rel_change=1.075505e-02
Iter 17 | step=0.0588 | rel_change=9.067087e-03
Iter 18 | step=0.0556 | rel_change=1.189779e-02
Iter 19 | step=0.0526 | rel_change=1.061307e-02
Iter 20 | step=0.0500 | rel_change=8.769234e-03
Iter 21 | step=0.0476 | rel_change=9.411